In [1]:
import os
import sys

parent_dir = os.path.abspath(os.path.join(os.pardir))
sys.path.append(parent_dir)

from libsql_graph_db import database as db
from dotenv import load_dotenv # type: ignore

load_dotenv()
db_url = os.getenv("LIBSQL_URL")
auth_token = os.getenv("LIBSQL_AUTH_TOKEN")


In [2]:
# To initialze the database and establish a connection with tursodb
db.initialize(db_url, auth_token)

Traceback (most recent call last):
thread '<unnamed>' panicked at /root/.cargo/registry/src/index.crates.io-6f17d22bba15001f/libsql-0.3.1/src/hrana/hyper.rs:88:9:
there is no reactor running, must be called from the context of a Tokio 1.x runtime
note: run with `RUST_BACKTRACE=1` environment variable to display a backtrace
  File "/home/anubhuti/Desktop/simple-graph-db/libsql_graph_db/database.py", line 79, in initialize
    return atomic(_init, db_url, auth_token)  # type: ignore
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
pyo3_runtime.PanicException: there is no reactor running, must be called from the context of a Tokio 1.x runtime


In [3]:
# To add a single node
def insert_single_node(db_url, auth_token, new_node, new_node_id):
    try:
        db.atomic(db.add_node(new_node, new_node_id), db_url, auth_token)
    except Exception as e:
        assert False
new_node = {'subject': 'MES', 'type': ['person', 'Dr']}
new_node_id = 1

insert_single_node(db_url, auth_token, new_node, new_node_id)
    

In [4]:
# To add multiple nodes
def bulk_insert_operations(db_url, auth_token, nodes):
    ids = []
    bodies = []
    for id, body in nodes.items():
        ids.append(id)
        bodies.append(body)

    # bulk add and confirm
    db.atomic(db.add_nodes(bodies, ids), db_url, auth_token)

new_nodes = {
    2 : {'name': 'Peri', 'age': '90'},
    3 : {'name': 'Pema', 'age': '66'}
}

bulk_insert_operations(db_url, auth_token, new_nodes)

In [5]:
# To find for a node
def find_a_node(db_url, auth_token, node):
            return db.atomic(db.find_node(node), db_url, auth_token)


find_a_node(db_url, auth_token, 5)

Traceback (most recent call last):
thread '<unnamed>' panicked at /root/.cargo/registry/src/index.crates.io-6f17d22bba15001f/libsql-0.3.1/src/hrana/hyper.rs  File "/tmp/ipykernel_78534/3005601512.py", line 3, in find_a_node
:88:9:
there is no reactor running, must be called from the context of a Tokio 1.x runtime
pyo3_runtime.PanicException: there is no reactor running, must be called from the context of a Tokio 1.x runtime


{}

In [6]:
# For bulk upsert: Update and insert
def bulk_upsert(db_url, auth_token, nodes):
    ids = []
    bodies = []
    for id, body in nodes.items():
        ids.append(id)
        bodies.append(body)

    db.atomic(db.upsert_nodes(bodies, ids), db_url, auth_token)
nodes = {
    1 : {'name': 'Stanley', 'age': '30'},
    2 : {'name': 'Sheeran', 'type': ['singer', 'rich']}
}

bulk_upsert(db_url, auth_token, nodes)
    

In [7]:
# For single upsert: Update and insert a node
def single_upsert(db_url, auth_token, body, id):
    
    db.atomic(db.upsert_node(id, body), db_url, auth_token)

body = {'name': 'Sheeran', 'type': ['singer', 'rich']}
id = 2
single_upsert(db_url, auth_token, body, id)

In [8]:
# Bulk connect and confirm
def bulk_node_connect(db_url, auth_token, edges):
    sources = []
    targets = []
    properties = []
    for src, tgts in edges.items():
        for target in tgts:
            tgt, label = target
            sources.append(src)
            targets.append(tgt)
            if label:
                properties.append(label)
            else:
                properties.append({})
        

    db.atomic(db.connect_many_nodes(
        sources, targets, properties), db_url, auth_token)
    
edges = {
    2: [
        (3 , {'wealth':'1000 Billion'}),
        (2 , {'balance': '1000 rupees'})
    ]
}

bulk_node_connect(db_url, auth_token, edges)
    


In [9]:
# Single connect and confirm
def single_node_connect(db_url, auth_token, source, target,  property):
    db.atomic(db.connect_nodes(
        source, target,  property), db_url, auth_token)
    
source = 3
target = 1
property = {'relation': 'enemy'}

single_node_connect(db_url, auth_token, source, target, property)

In [10]:
# To remove nodes
def remove_bulk_nodes(db_url, auth_token, ids):
        db.atomic(db.remove_nodes(ids), db_url, auth_token)

ids = [1, 4]

remove_bulk_nodes(db_url, auth_token, ids)


In [11]:
# To remove a single node
def remove_single_node(db_url, auth_token, id):
    db.atomic(db.remove_node(id), db_url, auth_token)

remove_single_node(db_url, auth_token, 2)

In [12]:
# To traverse the nodes
def traverse_nodes(db_url, auth_token, src, tgt):
    return db.traverse(db_url, auth_token, src, tgt)
traverse_nodes(db_url, auth_token, 3, 2)

Traceback (most recent call last):
  File "/home/anubhuti/Desktop/simple-graph-db/libsql_graph_db/database.py", line 513, in traverse
thread '<unnamed>' panicked at /root/.cargo/registry/src/index.crates.io-6f17d22bba15001f/libsql-0.3.1/src/hrana/hyper.rs:88:9:
there is no reactor running, must be called from the context of a Tokio 1.x runtime
    return atomic(_traverse, db_url, auth_token)  # type: ignore
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
pyo3_runtime.PanicException: there is no reactor running, must be called from the context of a Tokio 1.x runtime


['3']

In [13]:
# To generate a query clause and find nodes
kv_name_like = db._generate_clause('name', predicate='LIKE')
print(db.atomic(db.find_nodes([kv_name_like], ('Pe%',)), db_url, auth_token))


Traceback (most recent call last):
  File "/tmp/ipykernel_78534/2893161704.py", line 3, in <module>
thread '<unnamed>' panicked at /root/.cargo/registry/src/index.crates.io-6f17d22bba15001f/libsql-0.3.1/src/hrana/hyper.rs:88:9:
there is no reactor running, must be called from the context of a Tokio 1.x runtime
pyo3_runtime.PanicException: there is no reactor running, must be called from the context of a Tokio 1.x runtime


[{'name': 'Pema', 'age': '66', 'id': 3}]


In [14]:
# Visualization
from libsql_graph_db import visualizers
raw_path = "temp_path" +  "sample1.dot"
visualizers.graphviz_visualize(db_url, auth_token, raw_path, ['3'])

TypeError: argument 'auth_token': 'function' object cannot be converted to 'PyString'

In [ ]:
# Visualization with bodies

raw_path = "temp_path" +  "bodies.dot"
path_with_bodies = db.traverse(db_url, auth_token, 2, with_bodies=True)
visualizers.graphviz_visualize_bodies(raw_path, path_with_bodies)

Traceback (most recent call last):
thread '<unnamed>' panicked at /root/.cargo/registry/src/index.crates.io-6f17d22bba15001f/libsql-0.3.1/src/hrana/hyper.rs  File "/home/anubhuti/Desktop/simple-graph-db/libsql_graph_db/database.py", line 491, in traverse
:88:9:
there is no reactor running, must be called from the context of a Tokio 1.x runtime
    return atomic(db_url, auth_token, _traverse)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
pyo3_runtime.PanicException: there is no reactor running, must be called from the context of a Tokio 1.x runtime
